In [34]:
# data manipulation
import numpy as np
import pandas as pd

#file processing / requests
from pathlib import Path
from tqdm import tqdm
import json
import urllib

#computer vision
import cv2
import torch
import torchvision

import PIL

#ML 
from sklearn.model_selection import train_test_split


#display
from IPython.display import display
from pylab import rcParams

#visualization
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import rc
import PIL.Image as Image

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.options.display.float_format = '{:,}'.format

In [ ]:
#from https://stats.oecd.org/Index.aspx?DataSetCode=ITF_INV-MTN_DATA

In [6]:
df = pd.read_csv('oecd_road_data.csv', sep = ',')
df.head()

,COUNTRY,Country,VARIABLE,Variable,MEASURE,Measure,YEAR,Year,Unit Code,Unit,PowerCode Code,PowerCode,Reference Period Code,Reference Period,Value,Flag Codes,Flags
0,ALB,Albania,I-MTN-RD,Road infrastructure maintenance,NAT,National currency,2000,2000,ALL,Lek,0,Units,NaN,NaN,"509,000,000.00",NaN,NaN
1,ALB,Albania,I-MTN-RD,Road infrastructure maintenance,NAT,National currency,2001,2001,ALL,Lek,0,Units,NaN,NaN,"733,000,000.00",NaN,NaN
2,ALB,Albania,I-MTN-RD,Road infrastructure maintenance,NAT,National currency,2002,2002,ALL,Lek,0,Units,NaN,NaN,"903,000,000.00",NaN,NaN
3,ALB,Albania,I-MTN-RD,Road infrastructure maintenance,NAT,National currency,2003,2003,ALL,Lek,0,Units,NaN,NaN,"793,000,000.00",NaN,NaN
4,ALB,Albania,I-MTN-RD,Road infrastructure maintenance,NAT,National currency,2004,2004,ALL,Lek,0,Units,NaN,NaN,"872,000,000.00",NaN,NaN


In [7]:
df.dtypes

COUNTRY                   object
Country                   object
VARIABLE                  object
Variable                  object
MEASURE                   object
Measure                   object
YEAR                       int64
Year                       int64
Unit Code                 object
Unit                      object
PowerCode Code             int64
PowerCode                 object
Reference Period Code    float64
Reference Period         float64
Value                     object
Flag Codes                object
Flags                     object
dtype: object

In [12]:
df.Variable.unique()

array(['Road infrastructure maintenance',
       'Airport infrastructure maintenance',
       'Rail infrastructure maintenance',
       'Inland waterway infrastucture maintenance',
       'Maritime port infrastructure maintenance',
       'Road infrastructure maintenance: Of which motorway'], dtype=object)

In [15]:
dfg = df.loc[(df['Variable'] == 'Road infrastructure maintenance')].groupby('Country')['Year'].max().reset_index()
print(dfg.Year.min())
dfg

2007


,Country,Year
0,Albania,2019
1,Armenia,2016
2,Austria,2019
3,Azerbaijan,2019
4,Belgium,2017
5,Bulgaria,2019
6,Canada,2019
7,Chile,2019
8,Croatia,2019
9,Czech Republic,2019


In [17]:
df.dtypes

COUNTRY                   object
Country                   object
VARIABLE                  object
Variable                  object
MEASURE                   object
Measure                   object
YEAR                       int64
Year                       int64
Unit Code                 object
Unit                      object
PowerCode Code             int64
PowerCode                 object
Reference Period Code    float64
Reference Period         float64
Value                     object
Flag Codes                object
Flags                     object
dtype: object

In [22]:
import re
df['Value'] = df['Value'].str.replace(',','')
df['Value'] = df['Value'].str.replace(' ','')
df['Value'] = df['Value'].str.replace('-','0')
df['Value'] = df['Value'].astype(float)
df['Value']

0        509000000.0
1        733000000.0
2        903000000.0
3        793000000.0
4        872000000.0
            ...     
10033    111016016.4
10034    119279297.1
10035    116535803.7
10036    122745044.4
10037    137000000.0
Name: Value, Length: 10038, dtype: float64

In [24]:
df

,COUNTRY,Country,VARIABLE,Variable,MEASURE,Measure,YEAR,Year,Unit Code,Unit,PowerCode Code,PowerCode,Reference Period Code,Reference Period,Value,Flag Codes,Flags
0,ALB,Albania,I-MTN-RD,Road infrastructure maintenance,NAT,National currency,2000,2000,ALL,Lek,0,Units,NaN,NaN,509000000.0,NaN,NaN
1,ALB,Albania,I-MTN-RD,Road infrastructure maintenance,NAT,National currency,2001,2001,ALL,Lek,0,Units,NaN,NaN,733000000.0,NaN,NaN
2,ALB,Albania,I-MTN-RD,Road infrastructure maintenance,NAT,National currency,2002,2002,ALL,Lek,0,Units,NaN,NaN,903000000.0,NaN,NaN
3,ALB,Albania,I-MTN-RD,Road infrastructure maintenance,NAT,National currency,2003,2003,ALL,Lek,0,Units,NaN,NaN,793000000.0,NaN,NaN
4,ALB,Albania,I-MTN-RD,Road infrastructure maintenance,NAT,National currency,2004,2004,ALL,Lek,0,Units,NaN,NaN,872000000.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10033,NZL,New Zealand,I-MTN-RL,Rail infrastructure maintenance,NAT,National currency,2015,2015,NZD,New Zealand Dollar,0,Units,NaN,NaN,111016016.4,NaN,NaN
10034,NZL,New Zealand,I-MTN-RL,Rail infrastructure maintenance,NAT,National currency,2016,2016,NZD,New Zealand Dollar,0,Units,NaN,NaN,119279297.1,NaN,NaN
10035,NZL,New Zealand,I-MTN-RL,Rail infrastructure maintenance,NAT,National currency,2017,2017,NZD,New Zealand Dollar,0,Units,NaN,NaN,116535803.7,NaN,NaN
10036,NZL,New Zealand,I-MTN-RL,Rail infrastructure maintenance,NAT,National currency,2018,2018,NZD,New Zealand Dollar,0,Units,NaN,NaN,122745044.4,NaN,NaN


In [29]:
dfg = df.loc[(df['Variable'] == 'Road infrastructure maintenance') & (df['Year'] == 2016)].groupby(['Measure'])['COUNTRY'].count().reset_index()
pd.set_option('display.max_rows', df.shape[0]+1)
# print(dfg.Year.min())
dfg

,Measure,COUNTRY
0,Constant Euro,36
1,Euro,36
2,National currency,36


In [35]:
dfh = df.loc[(df['Variable'] == 'Road infrastructure maintenance') & (df['Year'] == 2016) & (df['Measure'] == 'Euro')].groupby(['COUNTRY'])['Value'].mean().reset_index()
pd.set_option('display.max_rows', df.shape[0]+1)
# print(dfg.Year.min())
dfh

,COUNTRY,Value
0,ALB,"13,004,791.24"
1,ARM,"11,586,346.99"
2,AUT,"697,000,000.0"
3,AZE,"18,702,181.92"
4,BEL,"528,000,000.0"
5,BGR,"174,864,505.6"
6,CAN,"4,885,097,151.0"
7,CHE,"2,699,504,678.0"
8,CHL,"830,897,547.8"
9,CZE,"767,272,565.9"


In [49]:
usd_eur_er_2016 = 1.11
oecd_countries_gdp_usd_2016 = 56098000000000 
oecd_countries_gdp_eur_2016 = oecd_countries_gdp_usd_2016 / 1.11 
oecd_spending_road_maintenance_eur_2016 = dfh['Value'].sum()
print(oecd_spending_road_maintenance_eur_2016 / usd_eur_er_2016)
perc_spend = oecd_spending_road_maintenance_2016 / oecd_countries_gdp_eur_2016
print(perc_spend)

88300484847.04504
0.0019393744408008166


In [51]:
print('Total OECD 2016 Road Spend as Percentage of GDP {}'.format(88300484847.04504, '{:,}'))
      #88,300,484,847.04504

Total OECD 2016 Road Spend as Percentage of GDP 88300484847.04504


In [48]:
us_spend_road_maintenance_usd_2016 = df.loc[(df['Variable'] == 'Road infrastructure maintenance') & (df['Year'] == 2016) & (df['Unit Code'] == 'USD') & (df['COUNTRY'] == 'USA')]\
['Value'].iloc[0]
us_gdp_2016_usd = 18750000000000
print(us_spend_road_maintenance_usd_2016/us_gdp_2016_usd)

0.0028484266666666666
